In [1]:
import sys
import numpy as np
import math
import scipy
import copy
import scipy.integrate as integrate
from scipy.integrate import quad
import collections
#import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
#from matplotlib import ticker, cm
#import seaborn as sns
from numpy import sqrt


#Header Codes

UnitCode = '164'
NodeCoordinates = '2411'
ElementList = '2412'
BoundaryConditions = '2467'

In [2]:
# reading the entire mesh .unv file into a List

dataList=[]
x = open("axi/mesh.unv","r")
for line in x:
    line=line.rstrip('\n')
    dataList.append(line.lstrip(' '))
dataList.append('end')
x.close()

#print(dataList)

In [3]:
# reading boundary condition file

boundList=[]
x = open("axi/boundarycondition.dat","r")
for line in x:
    line=line.rstrip('\n')
    boundList.append(line.lstrip(' '))
boundList.append('end')
x.close()

#print(boundList)

In [4]:
#initial condition and heat generation input
initialcond = float(boundList[1])
G = float(boundList[3])
boundarylist=[]
i=5
while boundList[i]!=('end'):
    if i < len(boundList):
        boundarylist.append(boundList[i])
        if(boundList[i+1]=='end'):
            i+=1
        else:
            i+=2
#print(boundarylist)

In [5]:
def check(text,linenumber):
    found = False
    if text in dataList[linenumber]:
        found = True
    return found
        
    
def findBlockStart(dataList,HeaderCode):
    linenumber=0
    for linenumber in range(len(dataList)):
        if check('-1',linenumber)== True:
            if check(HeaderCode,linenumber+1)==True:
                return(linenumber+2)

In [6]:
# code block to create NODE DICTIONARY

NodeDictionary  = {}
startLine = findBlockStart(dataList,NodeCoordinates)

line = copy.deepcopy(startLine)
coordinateLine = line+1

while dataList[line]!=('-1'):
    NodeDictionary.update({int((dataList[line].split())[0]):[float(x) for x in dataList[coordinateLine].split()]})
    line+=2
    coordinateLine+=2

print(NodeDictionary)

{1: [0.0, 0.0, 0.0], 2: [0.015, 0.0, 0.0], 3: [0.015, 0.015, 0.0], 4: [0.0, 0.015, 0.0], 5: [0.001875, 0.0, 0.0], 6: [0.00375, 0.0, 0.0], 7: [0.005625, 0.0, 0.0], 8: [0.0075, 0.0, 0.0], 9: [0.009375, 0.0, 0.0], 10: [0.01125, 0.0, 0.0], 11: [0.013125, 0.0, 0.0], 12: [0.015, 0.001875, 0.0], 13: [0.015, 0.00375, 0.0], 14: [0.015, 0.005625, 0.0], 15: [0.015, 0.0075, 0.0], 16: [0.015, 0.009375, 0.0], 17: [0.015, 0.01125, 0.0], 18: [0.015, 0.013125, 0.0], 19: [0.013125, 0.015, 0.0], 20: [0.01125, 0.015, 0.0], 21: [0.009375, 0.015, 0.0], 22: [0.0075, 0.015, 0.0], 23: [0.005625, 0.015, 0.0], 24: [0.00375, 0.015, 0.0], 25: [0.001875, 0.015, 0.0], 26: [0.0, 0.013125, 0.0], 27: [0.0, 0.01125, 0.0], 28: [0.0, 0.009375, 0.0], 29: [0.0, 0.0075, 0.0], 30: [0.0, 0.005625, 0.0], 31: [0.0, 0.00375, 0.0], 32: [0.0, 0.001875, 0.0], 33: [0.01195918598909752, 0.005937124545064841, 0.0], 34: [0.007175658029719823, 0.014102056223474265, 0.0], 35: [0.0026726179244138904, 0.007168404156594808, 0.0], 36: [0.0125

In [7]:
# code block to create Element Dictionary
EdgeElementDictionary = {}
ElementDictionary = {}
elementStart = findBlockStart(dataList,ElementList)

line = copy.deepcopy(elementStart)
i=1
while dataList[line]!=('-1'):
    if [int(x) for x in dataList[line].split()][5] == 2:
        EdgeElementDictionary.update({int((dataList[line].split())[0]):[int(x) for x in dataList[line+2].split()]})
        line+=3
    else:
        ElementDictionary.update({i:[int(x) for x in dataList[line+1].split()]})
        line+=2
        i+=1

#print(EdgeElementDictionary)
print(ElementDictionary)

{1: [47, 18, 3], 2: [81, 47, 36], 3: [47, 3, 19], 4: [48, 80, 20], 5: [21, 48, 20], 6: [78, 38, 106], 7: [20, 36, 19], 8: [24, 49, 23], 9: [49, 76, 23], 10: [34, 22, 23], 11: [37, 75, 24], 12: [25, 37, 24], 13: [74, 37, 50], 14: [50, 25, 4], 15: [34, 105, 22], 16: [35, 60, 123], 17: [61, 121, 123], 18: [68, 111, 122], 19: [100, 44, 122], 20: [72, 35, 121], 21: [69, 72, 121], 22: [73, 41, 120], 23: [72, 73, 120], 24: [51, 15, 16], 25: [33, 52, 119], 26: [38, 51, 16], 27: [51, 119, 52], 28: [77, 71, 119], 29: [106, 81, 118], 30: [51, 52, 15], 31: [80, 98, 118], 32: [17, 38, 16], 33: [88, 76, 117], 34: [75, 97, 117], 35: [94, 54, 116], 36: [54, 53, 33], 37: [76, 49, 117], 38: [68, 95, 111], 39: [82, 46, 87], 40: [2, 12, 56], 41: [71, 100, 116], 42: [2, 56, 11], 43: [58, 9, 40], 44: [61, 90, 115], 45: [99, 69, 115], 46: [101, 83, 114], 47: [84, 44, 114], 48: [82, 87, 113], 49: [104, 76, 113], 50: [57, 40, 9], 51: [107, 82, 112], 52: [108, 84, 112], 53: [55, 57, 11], 54: [57, 9, 10], 55: [5

In [8]:
# Read Boundaries
TempBDictionary={}
Convectbound={}
qbound={}
rbound={}
boundaryConditionStart = findBlockStart(dataList,BoundaryConditions)
line = copy.deepcopy(boundaryConditionStart)
while dataList[line]!=('-1'):
    btot = int((dataList[line].split())[7])
    bname = (dataList[line+1].split())[0]
    for i in range(len(boundarylist)):
        if bname == (boundarylist[i].split())[0]:
            if (boundarylist[i].split())[1] == 'Temp':
                for xi in range(int(btot/2+btot%2)-1):
                    k = line+1+xi+1
                    TempBDictionary.update({int((dataList[k].split())[1]):float((boundarylist[i].split())[2])})
                    TempBDictionary.update({int((dataList[k].split())[5]):float((boundarylist[i].split())[2])})
                if int(btot%2)==1:
                    j = line+1+int(btot/2+btot%2)
                    TempBDictionary.update({int((dataList[j].split())[1]):float((boundarylist[i].split())[2])})
                else:
                    j = line+1+int(btot/2+btot%2)
                    TempBDictionary.update({int((dataList[j].split())[1]):float((boundarylist[i].split())[2])})
                    TempBDictionary.update({int((dataList[j].split())[5]):float((boundarylist[i].split())[2])})
            if (boundarylist[i].split())[1] == 'Conv':
                for xi in range(int(btot/2+btot%2)-1):
                    k = line+1+xi+1
                    Convectbound.update({int((dataList[k].split())[1]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                    Convectbound.update({int((dataList[k].split())[5]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                if int(btot%2)==1:
                    j = line+1+int(btot/2+btot%2)
                    Convectbound.update({int((dataList[j].split())[1]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                else:
                    j = line+1+int(btot/2+btot%2)
                    Convectbound.update({int((dataList[j].split())[1]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                    Convectbound.update({int((dataList[j].split())[5]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
            if (boundarylist[i].split())[1] == 'Radi':
                for xi in range(int(btot/2+btot%2)-1):
                    k = line+1+xi+1
                    rbound.update({int((dataList[k].split())[1]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                    rbound.update({int((dataList[k].split())[5]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                if int(btot%2)==1:
                    j = line+1+int(btot/2+btot%2)
                    rbound.update({int((dataList[j].split())[1]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                else:
                    j = line+1+int(btot/2+btot%2)
                    rbound.update({int((dataList[j].split())[1]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
                    rbound.update({int((dataList[j].split())[5]):[float((boundarylist[i].split())[2]),float((boundarylist[i].split())[3])]})
            if (boundarylist[i].split())[1] == 'Flux':
                for xi in range(int(btot/2+btot%2)-1):
                    k = line+1+xi+1
                    qbound.update({int((dataList[k].split())[1]):float((boundarylist[i].split())[2])})
                    qbound.update({int((dataList[k].split())[5]):float((boundarylist[i].split())[2])})
                if int(btot%2)==1:
                    j = line+1+int(btot/2+btot%2)
                    qbound.update({int((dataList[j].split())[1]):float((boundarylist[i].split())[2])})
                else:
                    j = line+1+int(btot/2+btot%2)
                    qbound.update({int((dataList[j].split())[1]):float((boundarylist[i].split())[2])})
                    qbound.update({int((dataList[j].split())[5]):float((boundarylist[i].split())[2])})
    line+=int(btot/2+btot%2)+2
print(TempBDictionary)
#print(Convectbound)
#print(rbound)
print(qbound)

{}
{2: -20000.0, 3: -20000.0, 12: -20000.0, 13: -20000.0, 14: -20000.0, 15: -20000.0, 16: -20000.0, 17: -20000.0, 18: -20000.0}


#reading material dataA



In [9]:
materialList=[]
x = open("axi/material.dat","r")
for line in x:
    line=line.rstrip('\n')
    materialList.append(line.lstrip(' '))
materialList.append('end')
x.close()
density = float(materialList[1])
specificheat = float(materialList[3])
k = float(materialList[5])
print(density)
print(specificheat)
print(k)

7850.0
475.0
44.5


In [10]:
#calculate smaller element length
length=[]
for x in range(len(ElementDictionary)):
    length.append(math.sqrt((NodeDictionary[ElementDictionary[x+1][0]][0]-NodeDictionary[ElementDictionary[x+1][1]][0])**2+(NodeDictionary[ElementDictionary[x+1][0]][1]-NodeDictionary[ElementDictionary[x+1][1]][1])**2))
    length.append(math.sqrt((NodeDictionary[ElementDictionary[x+1][1]][0]-NodeDictionary[ElementDictionary[x+1][2]][0])**2+(NodeDictionary[ElementDictionary[x+1][1]][1]-NodeDictionary[ElementDictionary[x+1][2]][1])**2))
    length.append(math.sqrt((NodeDictionary[ElementDictionary[x+1][2]][0]-NodeDictionary[ElementDictionary[x+1][0]][0])**2+(NodeDictionary[ElementDictionary[x+1][2]][1]-NodeDictionary[ElementDictionary[x+1][0]][1])**2))
#calculate time step
alpha = k/(density*specificheat)
timestep = round(min(length)**2/(2*alpha), 2)
timestep

0.03

In [11]:
#Type of Analysis
analysis = 'axisymmetric'

#C, K and F Matrix with assembly
C = np.zeros((len(NodeDictionary),len(NodeDictionary)))
K = np.zeros((len(NodeDictionary),len(NodeDictionary)))
F = np.zeros((len(NodeDictionary),1))
nele = len(ElementDictionary)


#for loop for each element
for x in range(nele):
    #element area
    elemarea = abs((np.linalg.det(np.array([[1,NodeDictionary[ElementDictionary[x+1][0]][0],
                               NodeDictionary[ElementDictionary[x+1][0]][1]],
                              [1,NodeDictionary[ElementDictionary[x+1][1]][0],
                               NodeDictionary[ElementDictionary[x+1][1]][1]],
                              [1,NodeDictionary[ElementDictionary[x+1][2]][0],
                               NodeDictionary[ElementDictionary[x+1][2]][1]]])))/2)
    
    #shape function constants a,b and c
    ai = NodeDictionary[ElementDictionary[x+1][1]][0]*NodeDictionary[ElementDictionary[x+1][2]][1] - NodeDictionary[ElementDictionary[x+1][2]][0]*NodeDictionary[ElementDictionary[x+1][1]][1]
    bi = NodeDictionary[ElementDictionary[x+1][1]][1] - NodeDictionary[ElementDictionary[x+1][2]][1]
    ci = NodeDictionary[ElementDictionary[x+1][2]][0] - NodeDictionary[ElementDictionary[x+1][1]][0]
    aj = NodeDictionary[ElementDictionary[x+1][2]][0]*NodeDictionary[ElementDictionary[x+1][0]][1] - NodeDictionary[ElementDictionary[x+1][0]][0]*NodeDictionary[ElementDictionary[x+1][2]][1]
    bj = NodeDictionary[ElementDictionary[x+1][2]][1] - NodeDictionary[ElementDictionary[x+1][0]][1]
    cj = NodeDictionary[ElementDictionary[x+1][0]][0] - NodeDictionary[ElementDictionary[x+1][2]][0]
    ak = NodeDictionary[ElementDictionary[x+1][0]][0]*NodeDictionary[ElementDictionary[x+1][1]][1] - NodeDictionary[ElementDictionary[x+1][1]][0]*NodeDictionary[ElementDictionary[x+1][0]][1]
    bk = NodeDictionary[ElementDictionary[x+1][0]][1] - NodeDictionary[ElementDictionary[x+1][1]][1]
    ck = NodeDictionary[ElementDictionary[x+1][1]][0] - NodeDictionary[ElementDictionary[x+1][0]][0]
   
    #B, D and BTranspose Matrix and multiplication
    B = (1/(2*elemarea))*np.matrix([[bi,bj,bk], [ci,cj,ck]])
    D = np.array([[k,0],[0,k]])
    BTranspose = np.array(B).transpose()
    Bmult = np.matmul(np.matmul(BTranspose,D),B)
    
    if analysis == 'axisymmetric':
        #average r value
        raverage = (NodeDictionary[ElementDictionary[x+1][0]][0]+NodeDictionary[ElementDictionary[x+1][1]][0]+NodeDictionary[ElementDictionary[x+1][2]][0])/3
        Bmult = Bmult * 2 * np.pi * raverage
        
    # calculating the first half of the K matrix
    K1 = np.zeros(Bmult.shape)
    for i in range(list(Bmult.shape)[0]): 
        for j in range(list(Bmult.shape)[1]):
            K1[i,j]= list(quad(lambda x:(Bmult.item(i,j)),0,elemarea))[0]
    h=0  
    lij=0
    
    # calculating the second half of the K matrix
    K2 = np.zeros(Bmult.shape)
    for i in range(list(Bmult.shape)[0]): 
        for j in range(list(Bmult.shape)[1]):
            if ElementDictionary[x+1][i] in Convectbound:
                h = Convectbound[ElementDictionary[x+1][i]][0]
                if ElementDictionary[x+1][j] in Convectbound:
                    if i==j:
                        K2[i,j]= 2
                    else:
                        K2[i,j]= 1
                        # length of the element in convective boundary region
                        lij = np.sqrt((NodeDictionary[ElementDictionary[x+1][i]][0]-
                                       NodeDictionary[ElementDictionary[x+1][j]][0])**2+
                                      (NodeDictionary[ElementDictionary[x+1][i]][1]-
                                       NodeDictionary[ElementDictionary[x+1][j]][1])**2)
    Ke = K1+K2*h*lij/6
    
    #K matrix assembly
    for i in range(list(Ke.shape)[0]): # calculating the first half of the integral
        for j in range(list(Ke.shape)[1]):
            K[int(ElementDictionary[(x+1)][i])-1,int(ElementDictionary[(x+1)][j])-1]+=Ke[i,j]
    
    #C matrix
    ctemp = np.zeros((3,3))
    for i in range(3): 
        for j in range(3):
            if i==j:
                if analysis == 'axisymmetric':
                    ctemp[i,j]= 2*elemarea*density*specificheat*2*np.pi * raverage/12
                else:
                    ctemp[i,j]= 2*elemarea*density*specificheat/12                         
            else:
                if analysis == 'axisymmetric':
                    ctemp[i,j]= 1*elemarea*density*specificheat*2*np.pi * raverage/12
                else:
                    ctemp[i,j]= 1*elemarea*density*specificheat/12
            
    #C matrix assembly
    for l in range(3): # calculating the first half of the integral
        for m in range(3):
            C[int(ElementDictionary[(x+1)][l])-1,int(ElementDictionary[(x+1)][m])-1]+=ctemp[l,m]
    #print(C)
    
    #F matrix
    F1=(G*elemarea/3)*np.matrix([[1],[1],[1]])  #F1 is for internal heat generation part
    
    #F2 is for heat flux part
    F2=np.zeros((3,1))
    p1=np.zeros((3,1))
    F2p1 =[0,0]
    F2Lij=0
    j=0
    q=0
    h=0
    Ta=0
    for i in range(3):
        if ElementDictionary[x+1][i] in qbound:
            q = qbound[ElementDictionary[x+1][i]]
            F2p1[j]=ElementDictionary[x+1][i]
            p1[i][0]+=1
            j+=1
    if (F2p1[0]!=0)&(F2p1[1]!=0):
        F2lij = np.sqrt((NodeDictionary[F2p1[0]][0]-
                                      NodeDictionary[F2p1[1]][0])**2+
                                      (NodeDictionary[F2p1[0]][1]-
                                      NodeDictionary[F2p1[1]][1])**2)
        if analysis == 'axisymmetric':
            for i in range(3):
                if p1[i][0]==1:
                    p1[i][0]=2*NodeDictionary[ElementDictionary[x+1][i]][0]
            if (p1[0][0]>0) & (p1[1][0]>0): 
                p1[0][0]+=NodeDictionary[ElementDictionary[x+1][1]][0]
                p1[1][0]+=NodeDictionary[ElementDictionary[x+1][0]][0]
            elif (p1[1][0]>0) & (p1[2][0]>0):
                p1[1][0]+=NodeDictionary[ElementDictionary[x+1][2]][0]
                p1[2][0]+=NodeDictionary[ElementDictionary[x+1][1]][0]
            elif (p1[0][0]>0) & (p1[2][0]>0):
                p1[0][0]+=NodeDictionary[ElementDictionary[x+1][2]][0]
                p1[2][0]+=NodeDictionary[ElementDictionary[x+1][0]][0]
            F2=(2*np.pi*q*F2lij/6)*p1
        else:
            F2=(q*F2lij/2)*p1
    
    #F3 is for convective part
    F3=np.zeros((3,1))
    p1=np.zeros((3,1))
    F2p1 =[0,0]
    F2Lij=0
    m=0
    for i in range(3):
        if ElementDictionary[x+1][i] in Convectbound:
            h = Convectbound[ElementDictionary[x+1][i]][0]
            Ta = Convectbound[ElementDictionary[x+1][i]][1]
            F2p1[m]=ElementDictionary[x+1][i]
            p1[i][0]+=1
            m+=1
    if (F2p1[0]!=0)&(F2p1[1]!=0):
        F2lij = np.sqrt((NodeDictionary[F2p1[0]][0]-
                                      NodeDictionary[F2p1[1]][0])**2+
                                      (NodeDictionary[F2p1[0]][1]-
                                      NodeDictionary[F2p1[1]][1])**2)
        F3=(h*Ta*F2lij/2)*p1
    # Total Load matrix F
    Fe =F1+F2+F3
    
    #Assembly of F Matrix
    for i in range(list(Fe.shape)[0]):
        F[int(ElementDictionary[(x+1)][i])-1,0]+=Fe[i,0]
print(F)

[[ 0.        ]
 [-1.76714587]
 [-1.76714587]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-3.53429174]
 [-3.53429174]
 [-3.53429174]
 [-3.53429174]
 [-3.53429174]
 [-3.53429174]
 [-3.53429174]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.     

In [12]:
#Transient analysis for each time step to calculate temperature
timestep=1
steps=50
theta = 1
TMatFinal = []
Tem=np.zeros((len(NodeDictionary),1))
for i in range(len(Tem)):
    Tem[i-1]+=500
for x in range(steps+1):
    #print(Tem)
    A=np.matrix(C)-(1-theta)*timestep*np.matrix(K)
    RHS=(np.matmul(A,Tem))+timestep*(F)
    TMat=np.zeros((len(NodeDictionary),1))
    for j in NodeDictionary:
        if j in TempBDictionary:
            RHS[j-1][0]=TempBDictionary[j]
            TMat[j-1][0]=TempBDictionary[j]
    CTemp = np.matrix(C)+theta*timestep*K
    for z in range(len(TMat)):
        if TMat[z]!=0 :
            CTemp[z,:]=0
            CTemp[z,z]=1
    TMatFin = np.linalg.solve(CTemp,RHS)
    TMatFinal.append(TMatFin)
    Tem=TMatFin

In [15]:
f=open("axi/output.txt","w+")
for i in range(0,len(TMatFinal)):
    f.write(str(float(TMatFinal[i][70]))+"\n")
    print(TMatFinal[i][70])
f.close()

[[499.3459773]]
[[498.56991971]]
[[497.80295778]]
[[497.05540279]]
[[496.32135124]]
[[495.59541716]]
[[494.87416427]]
[[494.15557052]]
[[493.43847787]]
[[492.72223024]]
[[492.00645767]]
[[491.29095202]]
[[490.57559629]]
[[489.86032475]]
[[489.14510049]]
[[488.42990279]]
[[487.71471999]]
[[486.99954556]]
[[486.28437584]]
[[485.56920875]]
[[484.85404315]]
[[484.13887838]]
[[483.42371407]]
[[482.70855003]]
[[481.99338614]]
[[481.27822233]]
[[480.56305856]]
[[479.84789482]]
[[479.1327311]]
[[478.41756738]]
[[477.70240367]]
[[476.98723996]]
[[476.27207625]]
[[475.55691255]]
[[474.84174884]]
[[474.12658514]]
[[473.41142143]]
[[472.69625772]]
[[471.98109402]]
[[471.26593031]]
[[470.55076661]]
[[469.8356029]]
[[469.1204392]]
[[468.40527549]]
[[467.69011179]]
[[466.97494808]]
[[466.25978438]]
[[465.54462067]]
[[464.82945696]]
[[464.11429326]]
[[463.39912955]]


In [ ]:
#creating post process msh file for GiD
f=open("post.vtk","w+")
f.write("# vtk DataFile Version 2.0\n")
f.write("2D scalar data\n")
f.write("ASCII\n")
f.write("DATASET UNSTRUCTURED_GRID\n")
f.write("POINTS "+str(len(NodeDictionary))+" float\n")
for i in range(1,len(NodeDictionary)+1):
    f.write(str(NodeDictionary[i][0])+" "+str(NodeDictionary[i][1])+" "+str(NodeDictionary[i][2])+"\n")
f.write("CELLS "+str(len(ElementDictionary))+" "+str(len(ElementDictionary)*4)+"\n")
for i in range(1,len(ElementDictionary)+1):
    f.write("3 "+str(int(ElementDictionary[i][0])-1)+" "+str(int(ElementDictionary[i][1])-1)+" "+str(int(ElementDictionary[i][2])-1)+"\n")
f.write("CELL_TYPES "+str(len(ElementDictionary))+"\n")
for i in range(1,len(ElementDictionary)+1):
    f.write("5\n")
for i in range(1,len(TMatFinal)):
    f.write("POINT_DATA "+str(len(NodeDictionary))+"\n")
    f.write("SCALARS temperature float 1\n")
    f.write("LOOKUP_TABLE default\n")
    for j in range(0,len(NodeDictionary)):
        f.write(str(TMatFinal[i][j][0])+" ")
f.close()

In [43]:
str(float(TMatFinal[0][70]))

'499.34597730435877'